# LangGraph use human-in-loop


## Import Models


1. 导入相关的库、模块


In [1]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from dotenv import load_dotenv

load_dotenv()

True

### 定义 LangGraph 状态图


In [2]:
# from langchain.chat_models import init_chat_model
import os
from langchain_openai import ChatOpenAI

# llm = init_chat_model('deepseek:deepseek-chat')
llm = ChatOpenAI(
  model='Qwen/Qwen3-30B-A3B-Instruct-2507',
  openai_api_key=os.environ['SILICONFLOW_API_KEY'],
  openai_api_base='https://api.siliconflow.cn/v1',
  streaming=True,
)


class State(TypedDict):
  messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


@tool
def human_assistance(query: str) -> str:
  """Request assistance from a human."""
  human_response = interrupt({'query': query})
  return human_response['data']


tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
  message = llm_with_tools.invoke(state['messages'])
  assert len(message.tool_calls) <= 1
  return {'messages': [message]}


graph_builder.add_node('chatbot', chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node('tools', tool_node)

graph_builder.add_conditional_edges(
  'chatbot',
  tools_condition,
)
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')
graph_builder.add_edge('chatbot', END)

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [3]:
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

## Prompt the chatbot

### 现在，向聊天机器人提出一个问题，以激活新的 human_assistance 工具


In [4]:
user_input = '我需要一些专家指导来构建人工智能代理。你能帮我请求帮助吗？'
config = {'configurable': {'thread_id': '1'}}

events = graph.stream(
  {'messages': [{'role': 'user', 'content': user_input}]},
  config,
  stream_mode='values',
)
for event in events:
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================ Human Message =================================

我需要一些专家指导来构建人工智能代理。你能帮我请求帮助吗？
================================== Ai Message ==================================
Tool Calls:
  human_assistance (019896dc6a809e9b38421b71dd774db6)
 Call ID: 019896dc6a809e9b38421b71dd774db6
  Args:
    query: 关于如何构建人工智能代理的专家指导，包括架构设计、关键技术选择、开发流程和最佳实践等方面的建议。


聊天机器人生成了一个工具调用，但随后执行被中断。如果你检查图形状态，会发现它在工具节点处停止：


In [5]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

## Resume execution

要继续执行，请传递一个包含工具所需数据的 Command 对象。该数据的格式可根据实际需求进行自定义。

对于这个示例，使用一个键为 "data" 的字典：


In [6]:
human_response = '我们，专家在这里提供帮助！我们建议您查看LangGraph以构建您的代理。它比简单的自主代理更可靠、更可扩展。'

human_command = Command(resume={'data': human_response})

events = graph.stream(human_command, config, stream_mode='values')
for event in events:
  if 'messages' in event:
    event['messages'][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (019896dc6a809e9b38421b71dd774db6)
 Call ID: 019896dc6a809e9b38421b71dd774db6
  Args:
    query: 关于如何构建人工智能代理的专家指导，包括架构设计、关键技术选择、开发流程和最佳实践等方面的建议。
================================= Tool Message =================================
Name: human_assistance

我们，专家在这里提供帮助！我们建议您查看LangGraph以构建您的代理。它比简单的自主代理更可靠、更可扩展。
================================== Ai Message ==================================

非常感谢专家的指导！根据您提供的建议，我将为您简要介绍如何使用 **LangGraph** 构建可靠且可扩展的人工智能代理。

---

### 🚀 为什么选择 LangGraph？
LangGraph 是一个基于图结构的框架，专为构建复杂的、可协作的 AI 代理而设计。与简单的“自主代理”相比，LangGraph 的优势包括：

- ✅ **可扩展性**：支持多步骤推理、循环决策和动态流程。
- ✅ **可解释性**：通过可视化图结构清晰展示代理的决策路径。
- ✅ **灵活性**：可轻松集成多个 LLM、工具、记忆系统和条件逻辑。
- ✅ **可靠性**：避免无限循环和逻辑错误，支持超时、错误处理和恢复机制。

---

### 🔧 构建 AI 代理的关键步骤（基于 LangGraph）

#### 1. **明确代理的目标与角色**
   - 例如：你想要一个“项目管理助手”代理，能制定计划、分配任务、监控进度。
   - 定义其核心能力：任务分解、时间估算、提醒、与人沟通等。

#### 2. **设计代理的图结构**
   使用 LangGraph 构建一个有向图（DAG 

输入已接收并作为工具消息处理。请查看此调用的 LangSmith 跟踪记录，以查看上述调用中执行的具体操作。请注意，在第一步中已加载状态，以便我们的聊天机器人能够从上次中断处继续。

恭喜！您已使用 interrupt 为聊天机器人添加了“人类在循环中”执行功能，这使得在必要时可实现人类监督与干预。此功能为您的 AI 系统开辟了更多可能的用户界面设计空间。由于您已添加了检查点，只要底层持久化层仍在运行，图形可被无限暂停，并在任何时间点恢复执行，且恢复后系统状态与暂停前完全一致。
